<h2 align="center">点击下列图标在线运行HanLP</h2>
<div align="center">
	<a href="https://colab.research.google.com/github/hankcs/HanLP/blob/doc-zh/plugins/hanlp_demo/hanlp_demo/zh/tok_stl.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
	<a href="https://mybinder.org/v2/gh/hankcs/HanLP/doc-zh?filepath=plugins%2Fhanlp_demo%2Fhanlp_demo%2Fzh%2Ftok_stl.ipynb" target="_blank"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>
</div>

## 安装

无论是Windows、Linux还是macOS，HanLP的安装只需一句话搞定：

In [ ]:
!pip install hanlp -U

## 加载模型
HanLP的工作流程是先加载模型，模型的标示符存储在`hanlp.pretrained`这个包中，按照NLP任务归类。

In [3]:
import hanlp
hanlp.pretrained.tok.ALL # 语种见名称最后一个字段或相应语料库

ModuleNotFoundError: No module named 'hanlp'

调用`hanlp.load`进行加载，模型会自动下载到本地缓存。自然语言处理分为许多任务，分词只是最初级的一个。

In [2]:
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
tok

### 进阶知识
你可以通过加载不同的模型实现各种颗粒度、各种分词标准、各种领域的中文分词。其中，coarse和fine模型训练自`9970`万字的大型综合语料库，覆盖新闻、社交媒体、金融、法律等多个领域，是已知范围内**全世界最大**的中文分词语料库。语料库规模决定实际效果，面向生产环境的语料库应当在千万字量级。欢迎用户在自己的语料上[训练或微调模型](https://github.com/hankcs/HanLP/tree/master/plugins/hanlp_demo/hanlp_demo/zh/train)以适应新领域。语料库标注标准决定最终的分词标准，模型的准确率决定多大程度上再现该分词标准。更多背景知识请参考[《自然语言处理入门》](http://nlp.hankcs.com/book.php)。

## 执行分词

In [3]:
tok(['商品和服务。', '阿婆主来到北京立方庭参观自然语义科技公司'])

[['商品', '和', '服务', '。'], ['阿婆主', '来到', '北京立方庭', '参观', '自然语义科技公司']]

### 细分标准

你可以通过加载`FINE_ELECTRA_SMALL_ZH`模型实现细粒度中文分词：

In [4]:
tok_fine = hanlp.load(hanlp.pretrained.tok.FINE_ELECTRA_SMALL_ZH)

无论哪个模型，分词器的接口是完全一致的：

In [5]:
tok_fine('阿婆主来到北京立方庭参观自然语义科技公司')

['阿婆', '主', '来到', '北京', '立方庭', '参观', '自然', '语义', '科技', '公司']

### 无限长度
众所周知，Transformer的输入有长度限制（通常是512）。幸运地是，HanLP的滑动窗口技巧完美地突破了该限制。只要你的内存（显存）足够，HanLP就可以处理无限长的句子。

### 并行分词
无论是CPU还是GPU，同时传入多个句子都将并行分词。也就是说，仅花费1个句子的时间可以处理多个句子。然而工作研究中的文本通常是一篇文档，而不是许多句子。此时可以利用HanLP提供的分句功能和流水线模式优雅应对，既能处理长文本又能并行化。只需创建一个流水线`pipeline`，第一级管道分句，第二级管道分词：

In [6]:
HanLP = hanlp.pipeline() \
    .append(hanlp.utils.rules.split_sentence) \
    .append(tok)
HanLP('量体裁衣，HanLP提供RESTful和native两种API。两者在语义上保持一致，在代码上坚持开源。')

[['量体裁衣', '，', 'HanLP', '提供', 'RESTful', '和', 'native', '两', '种', 'API', '。'],
 ['两者', '在', '语义', '上', '保持', '一致', '，', '在', '代码', '上', '坚持', '开源', '。']]

返回结果是每个句子的分词`list`，如果要将它们合并到一个`list`里该怎么办呢？聪明的用户可能已经想到了，再加一级`lambda`管道：

In [7]:
HanLP.append(lambda sents: sum(sents, []))
print(HanLP('量体裁衣，HanLP提供RESTful和native两种API。两者在语义上保持一致，在代码上坚持开源。'))

['量体裁衣', '，', 'HanLP', '提供', 'RESTful', '和', 'native', '两', '种', 'API', '。', '两者', '在', '语义', '上', '保持', '一致', '，', '在', '代码', '上', '坚持', '开源', '。']


## 自定义词典
自定义词典为分词任务的成员变量：

In [8]:
tok.dict_combine, tok.dict_force

(None, None)

HanLP支持合并和强制两种优先级的自定义词典，以满足不同场景的需求。

不挂词典：

In [9]:
tok.dict_force = tok.dict_combine = None
tok("商品和服务项目")

['商品', '和', '服务', '项目']

### 强制模式
强制模式优先输出正向最长匹配到的自定义词条（慎用，详见[《自然语言处理入门》](http://nlp.hankcs.com/book.php)第二章）：

In [10]:
tok.dict_force = {'和服', '服务项目'}
tok("商品和服务项目")

['商品', '和服', '务', '项目']

与大众的朴素认知不同，词典优先级最高未必是好事，极有可能匹配到不该分出来的自定义词语，导致歧义。自定义词语越长，越不容易发生歧义。这启发我们将强制模式拓展为强制校正功能。

强制校正原理相似，但会将匹配到的自定义词条替换为相应的分词结果:

In [11]:
tok.dict_force = {'和服务': ['和', '服务']}
tok("正向匹配商品和服务、任何和服务必按上述切分")

['正向', '匹配', '商品', '和', '服务', '、', '任何', '和', '服务', '必', '按', '上述', '切分']

### 合并模式
合并模式的优先级低于统计模型，即`dict_combine`会在统计模型的分词结果上执行最长匹配并合并匹配到的词条。一般情况下，推荐使用该模式。

In [12]:
tok.dict_force = None
tok.dict_combine = {'和服', '服务项目'}
tok("商品和服务项目")

['商品', '和', '服务项目']

需要算法基础才能理解，初学者可参考[《自然语言处理入门》](http://nlp.hankcs.com/book.php)。

#### 空格单词

含有空格、制表符等（Transformer tokenizer去掉的字符）的词语需要用`tuple`的形式提供：

In [13]:
tok.dict_combine = {('iPad', 'Pro'), '2个空格'}
tok("如何评价iPad Pro ？iPad  Pro有2个空格")

['如何', '评价', 'iPad Pro', '？', 'iPad  Pro', '有', '2个空格']

聪明的用户请继续阅读，`tuple`词典中的字符串其实等价于该字符串的所有可能的切分方式：

In [14]:
dict(tok.dict_combine.config["dictionary"]).keys()

dict_keys([('2', '个', '空', '格'), ('2', '个', '空格'), ('2', '个空格'), ('2', '个空', '格'), ('2个', '空', '格'), ('2个', '空格'), ('2个空格',), ('2个空', '格'), ('iPad', 'Pro')])

## 单词位置

HanLP支持输出每个单词在文本中的原始位置，以便用于搜索引擎等场景。在词法分析中，非语素字符（空格、换行、制表符等）会被剔除，此时需要额外的位置信息才能定位每个单词：

In [15]:
tok.config.output_spans = True
sent = '2021 年\nHanLPv2.1 为生产环境带来次世代最先进的多语种NLP技术。'
word_offsets = tok(sent)
print(word_offsets)

[['2021', 0, 4], ['年', 5, 6], ['HanLPv2.1', 7, 16], ['为', 17, 18], ['生产', 18, 20], ['环境', 20, 22], ['带来', 22, 24], ['次', 24, 25], ['世代', 25, 27], ['最', 27, 28], ['先进', 28, 30], ['的', 30, 31], ['多', 31, 32], ['语种', 32, 34], ['NLP', 34, 37], ['技术', 37, 39], ['。', 39, 40]]


返回格式为三元组（单词，单词的起始下标，单词的终止下标），下标以字符级别计量。

In [16]:
for word, begin, end in word_offsets:
    assert word == sent[begin:end]


In [2]:
print('how')

how
